# Purpose
This notebook demonstrates validation and invalidation of the following entities in Brick.
- Building
- Room

In [1]:
from pyshacl import validate
from rdflib import Namespace, Literal, XSD

from mason import BrickClassGenerator, compile_model, Brick

import tasty.constants as tc
import tasty.graphs as tg
from tasty.shapes_loader import ShapesLoader

# Setup

In [2]:
# pass or fail setting
fail = False

# print graphs setting
print_graphs = True

# entity propterties
year_built = 2021
primary_function = 'Adult Education'
thermal_transmittance = 2021.0
area = 2021.0
volume = 2021.0

if fail:
    year_built = str(year_built)
    primary_function = primary_function + '_fail'
    thermal_transmittance = str(thermal_transmittance)
    area = str(area)
    volume = str(volume)

# namespace
EX = Namespace('urn:example#')

# Data Graph (oomason)

## Building

In [3]:
# entity
building = Brick.Building(EX["building"], "Building")

# shapes
year_built_shape = Brick.EntityProperty.YearBuiltShape(year_built)
primary_function_shape = Brick.EntityProperty.BuildingPrimaryFunctionShape(Literal(primary_function, datatype=XSD.string)) # newer RDFlib is more specific about some datatypes
thermal_transmittance_shape = Brick.EntityProperty.ThermalTransmittenceShape(thermal_transmittance, Brick.Unit.Watt_per_Square_Meter_Kelvin) # transmittence misspelled, check IP units

# add shapes to entity
building.add_yearBuilt(year_built_shape)
building.add_buildingPrimaryFunction(primary_function_shape)
building.add_thermalTransmittence(thermal_transmittance_shape)

## Room

In [4]:
# entity
room = Brick.Room(EX['room1'], 'Room 1')

# shapes
area_shape = Brick.EntityProperty.AreaShape(area, Brick.Unit.Square_Foot)
volume_shape = Brick.EntityProperty.VolumeShape(volume, Brick.Unit.Cubic_Foot)

# add shapes to entity
room.add_area(area_shape)
room.add_volume(volume_shape)

In [5]:
# compile data graph, which validates it too
data_graph = compile_model([('ex', EX)]) # provide namespaces when compiling

# print
if print_graphs: tg.print_graph(data_graph)

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix ex: <urn:example#> .
@prefix unit: <http://qudt.org/vocab/unit/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:building a brick:Building ;
    brick:buildingPrimaryFunction [ a brick:BuildingPrimaryFunctionShape ;
            brick:value "Adult Education"^^xsd:string ] ;
    brick:thermalTransmittence [ a brick:ThermalTransmittenceShape ;
            brick:hasUnit unit:W-PER-M2-K ;
            brick:value 2.021e+03 ] ;
    brick:yearBuilt [ a brick:YearBuiltShape ;
            brick:value 2021 ] .

ex:room1 a brick:Room ;
    brick:area [ a brick:AreaShape ;
            brick:hasUnit unit:FT2 ;
            brick:value 2.021e+03 ] ;
    brick:volume [ a brick:VolumeShape ;
            brick:hasUnit unit:FT3 ;
            brick:value 2.021e+03 ] .




# Shapes Graph (tasty)

In [6]:
schema = tc.BRICK
version = tc.V1_2

# load ontology
brick_ontology = tg.load_ontology(schema, version)

# shapes graph
sl = ShapesLoader(schema)
shapes_graph = sl.load_all_shapes()
shapes_graph.bind('ex', EX)

# print
if print_graphs: tg.print_graph(shapes_graph)

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix unit: <http://qudt.org/vocab/unit/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

brick:AggregationShape a owl:Class,
        sh:NodeShape ;
    sh:property [ a sh:PropertyShape ;
            skos:definition "Interval expressed in an ISO 8601 Duration string, e.g. RP1D" ;
            sh:datatype xsd:string ;
            sh:minCount 1 ;
            sh:path brick:aggregationInterval ],
        [ a sh:PropertyShape ;
            skos:definition "The aggregation function applied to data in the interval which produces the value" ;
            sh:in ( "max" "min" "count" "mean" "sum" "median" "mode" ) ;
            sh:minCount 1 ;
            sh:path brick:aggregationFunction ] .

brick:AreaShape a owl:Class,
      

# Validate (RDFlib)

In [7]:
conforms, results_graph, results_text = validate(data_graph=data_graph, shacl_graph=shapes_graph, ont_graph=brick_ontology)
print(results_text)

Validation Report
Conforms: True



# TODO
- [ ] incorporate oomason into tasty
- [ ] automate `generated_shapes/brick_all.ttl`